# TrashCAN Dataset YOLOv8 Training

This notebook trains a YOLOv8 model on the TrashCAN dataset for underwater trash detection.

## Dataset Info:
- **22 classes** (14 trash classes + 8 marine life/other)
- **6,065 training images**
- **1,147 validation images**
- **Trash classes**: bag, bottle, can, container, clothing, cup, net, pipe, rope, snack_wrapper, tarp, branch, wreckage, unknown_instance

## 1. Install Dependencies

In [7]:
# Install required packages
!pip install ultralytics
!pip install torch torchvision
!pip install opencv-python
!pip install matplotlib
!pip install pillow

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip


## 2. Import Libraries and Setup

In [12]:
import os
import torch
import matplotlib.pyplot as plt
from ultralytics import YOLO
from pathlib import Path
import yaml

# Check device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")
print(f"PyTorch version: {torch.__version__}")

# Set working directory
os.chdir('/home/imtiyaz/Desktop/MarineLife-and-Pollution')
print(f"Working directory: {os.getcwd()}")

Using device: cpu
PyTorch version: 2.0.1+cpu
Working directory: /home/imtiyaz/Desktop/MarineLife-and-Pollution


## 3. Verify Dataset Structure

In [13]:
# Check dataset structure
data_yaml_path = "transcan_dataset/yolo_format/data.yaml"

# Load and display data.yaml
with open(data_yaml_path, 'r') as f:
    data_config = yaml.safe_load(f)
    
print("Dataset Configuration:")
print(f"Number of classes: {data_config['nc']}")
print(f"Class names: {data_config['names']}")
print(f"Train path: {data_config['train']}")
print(f"Val path: {data_config['val']}")

# Check if paths exist
train_images = "transcan_dataset/yolo_format/images/train"
val_images = "transcan_dataset/yolo_format/images/val"
train_labels = "transcan_dataset/yolo_format/labels/train"
val_labels = "transcan_dataset/yolo_format/labels/val"

print(f"\nDataset Statistics:")
print(f"Train images: {len(os.listdir(train_images))}")
print(f"Train labels: {len(os.listdir(train_labels))}")
print(f"Val images: {len(os.listdir(val_images))}")
print(f"Val labels: {len(os.listdir(val_labels))}")

Dataset Configuration:
Number of classes: 22
Class names: ['rov', 'plant', 'animal_fish', 'animal_starfish', 'animal_shells', 'animal_crab', 'animal_eel', 'animal_etc', 'trash_clothing', 'trash_pipe', 'trash_bottle', 'trash_bag', 'trash_snack_wrapper', 'trash_can', 'trash_cup', 'trash_container', 'trash_unknown_instance', 'trash_branch', 'trash_wreckage', 'trash_tarp', 'trash_rope', 'trash_net']
Train path: images/train
Val path: images/val

Dataset Statistics:
Train images: 5936
Train labels: 5936
Val images: 1147
Val labels: 1147


## 4. Initialize YOLOv8 Model

In [14]:
# Initialize YOLOv8 model
model = YOLO('yolov8n.pt')  # nano model for faster training

# You can also try:
# model = YOLO('yolov8s.pt')  # small model (better accuracy, slower)
# model = YOLO('yolov8m.pt')  # medium model (even better accuracy, much slower)

print(f"Model loaded: {model.model}")

Model loaded: DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C2f(
      (cv1): Conv(
        (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True

## 5. Training Configuration

In [15]:
# Create output directory
output_dir = "models/TrashCAN_YoloV8_Jupyter"
os.makedirs(output_dir, exist_ok=True)

# Training parameters
training_config = {
    'data': data_yaml_path,
    'epochs': 50,  # Reduced for faster training in Jupyter
    'imgsz': 640,
    'batch': 8,   # Reduced batch size for stability
    'device': device,
    'project': output_dir,
    'name': 'trashcan_detection',
    'save_period': 5,  # Save every 5 epochs
    'patience': 10,    # Early stopping
    'workers': 2,      # Reduced workers
    'optimizer': 'AdamW',
    'lr0': 0.01,
    'weight_decay': 0.0005,
    'warmup_epochs': 3,
    'box': 7.5,
    'cls': 0.5,
    'dfl': 1.5,
    'verbose': True,
    'plots': True
}

print("Training Configuration:")
for key, value in training_config.items():
    print(f"  {key}: {value}")

Training Configuration:
  data: transcan_dataset/yolo_format/data.yaml
  epochs: 50
  imgsz: 640
  batch: 8
  device: cpu
  project: models/TrashCAN_YoloV8_Jupyter
  name: trashcan_detection
  save_period: 5
  patience: 10
  workers: 2
  optimizer: AdamW
  lr0: 0.01
  weight_decay: 0.0005
  warmup_epochs: 3
  box: 7.5
  cls: 0.5
  dfl: 1.5
  verbose: True
  plots: True


## 6. Start Training

In [16]:
# Start training
print("Starting YOLOv8 training on TrashCAN dataset...")
print("This will take several hours depending on your hanrdware.")
print("You can monitor progress in the output below.")

# Train the model
results = model.train(**training_config)

Starting YOLOv8 training on TrashCAN dataset...
This will take several hours depending on your hanrdware.
You can monitor progress in the output below.
New https://pypi.org/project/ultralytics/8.3.229 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.127 🚀 Python-3.10.12 torch-2.0.1+cpu CPU (11th Gen Intel Core(TM) i5-1155G7 2.50GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=transcan_dataset/yolo_format/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf

train: Scanning /home/imtiyaz/Desktop/MarineLife-and-Pollution/transcan_dataset/yolo_format/labels/train.cache... 5936 images, 0 backgrounds, 0 corrupt: 

val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 130.6±27.2 MB/s, size: 26.1 KB)



val: Scanning /home/imtiyaz/Desktop/MarineLife-and-Pollution/transcan_dataset/yolo_format/labels/val.cache... 1147 images, 0 backgrounds, 0 corrupt: 100%

val: /home/imtiyaz/Desktop/MarineLife-and-Pollution/transcan_dataset/yolo_format/images/val/vid_000143_frame0000013.jpg: 1 duplicate labels removed


...

                   all       1147       2587      0.573     0.0322     0.0263     0.0128

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       2/50         0G      1.795      3.169      1.804         29        640: 100%|██████████| 742/742 [24:53<00:00,  2.01s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 72/72 [01:13<00:00,  1.03s/it]


                   all       1147       2587      0.493      0.097     0.0503     0.0288

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       3/50         0G      1.712      2.985       1.73         18        640:  58%|█████▊    | 431/742 [14:43<10:37,  2.05s/it]
Exception ignored in: <function tqdm.__del__ at 0x7575f1663d90>
Traceback (most recent call last):
  File "/home/imtiyaz/.local/lib/python3.10/site-packages/tqdm/std.py", line 1147, in __del__
    def __del__(self):
KeyboardInterrupt: 


KeyboardInterrupt: 

## 7. Validate the Trained Model

In [ ]:
# Validate the model
print("Validating the trained model...")
validation_results = model.val()

print(f"Validation Results:")
print(f"mAP50: {validation_results.box.map50:.4f}")
print(f"mAP50-95: {validation_results.box.map:.4f}")

## 8. Display Training Results

In [ ]:
# Display training plots
results_dir = f"{output_dir}/trashcan_detection"

# Load and display results.png if it exists
results_plot = f"{results_dir}/results.png"
if os.path.exists(results_plot):
    from PIL import Image
    img = Image.open(results_plot)
    plt.figure(figsize=(15, 10))
    plt.imshow(img)
    plt.axis('off')
    plt.title('Training Results')
    plt.show()
else:
    print(f"Results plot not found at {results_plot}")

# Display confusion matrix
confusion_matrix = f"{results_dir}/confusion_matrix.png"
if os.path.exists(confusion_matrix):
    img = Image.open(confusion_matrix)
    plt.figure(figsize=(12, 10))
    plt.imshow(img)
    plt.axis('off')
    plt.title('Confusion Matrix')
    plt.show()
else:
    print(f"Confusion matrix not found at {confusion_matrix}")

## 9. Test the Trained Model

In [ ]:
# Load the best trained model
best_model_path = f"{results_dir}/weights/best.pt"

if os.path.exists(best_model_path):
    # Load the trained model
    trained_model = YOLO(best_model_path)
    
    # Test on a sample image
    test_image_dir = "transcan_dataset/yolo_format/images/val"
    test_images = os.listdir(test_image_dir)[:5]  # Test on first 5 images
    
    for img_name in test_images:
        img_path = os.path.join(test_image_dir, img_name)
        
        # Run inference
        results = trained_model(img_path)
        
        # Display results
        for result in results:
            # Plot the result
            result.show()
            
            # Print detections
            if result.boxes is not None:
                print(f"\nDetections in {img_name}:")
                for box in result.boxes:
                    class_id = int(box.cls[0])
                    confidence = float(box.conf[0])
                    class_name = data_config['names'][class_id]
                    print(f"  - {class_name}: {confidence:.3f}")
            else:
                print(f"No detections in {img_name}")
else:
    print(f"Trained model not found at {best_model_path}")
    print("Make sure training completed successfully.")

## 10. Create Trash-Only Detection Function

In [ ]:
# Create trash-only detector class
class TrashOnlyDetector:
    def __init__(self, model_path):
        self.model = YOLO(model_path)
        
        # TrashCAN class names
        self.all_classes = [
            'rov', 'plant', 'animal_fish', 'animal_starfish', 'animal_shells',
            'animal_crab', 'animal_eel', 'animal_etc', 'trash_clothing', 'trash_pipe',
            'trash_bottle', 'trash_bag', 'trash_snack_wrapper', 'trash_can', 'trash_cup',
            'trash_container', 'trash_unknown_instance', 'trash_branch', 'trash_wreckage',
            'trash_tarp', 'trash_rope', 'trash_net'
        ]
        
        # Only trash classes (indices 8-21)
        self.trash_class_indices = [8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
        self.trash_classes = [self.all_classes[i] for i in self.trash_class_indices]
        
    def detect_trash_only(self, image_path, conf_threshold=0.5):
        """Detect only trash objects in the image"""
        results = self.model(image_path)
        
        trash_detections = []
        
        for result in results:
            boxes = result.boxes
            if boxes is not None:
                for box in boxes:
                    class_id = int(box.cls[0])
                    confidence = float(box.conf[0])
                    
                    # Only keep trash detections
                    if class_id in self.trash_class_indices and confidence >= conf_threshold:
                        trash_detections.append({
                            'class_id': class_id,
                            'class_name': self.all_classes[class_id],
                            'confidence': confidence,
                            'bbox': box.xyxy[0].tolist()  # [x1, y1, x2, y2]
                        })
        
        return trash_detections

# Test trash-only detection
if os.path.exists(best_model_path):
    trash_detector = TrashOnlyDetector(best_model_path)
    
    # Test on a sample image
    test_img = os.path.join(test_image_dir, test_images[0])
    trash_detections = trash_detector.detect_trash_only(test_img, conf_threshold=0.3)
    
    print(f"\nTrash-only detections in {test_images[0]}:")
    if trash_detections:
        for detection in trash_detections:
            print(f"  - {detection['class_name']}: {detection['confidence']:.3f}")
    else:
        print("  No trash detected")
else:
    print("Model not trained yet. Run the training cells first.")

## 11. Export Model (Optional)

In [ ]:
# Export model to different formats
if os.path.exists(best_model_path):
    export_model = YOLO(best_model_path)
    
    # Export to ONNX format (for deployment)
    export_model.export(format='onnx')
    print("Model exported to ONNX format")
    
    # Export to TensorRT (if you have NVIDIA GPU)
    # export_model.export(format='engine')
    
    print(f"\nTrained model location: {best_model_path}")
    print(f"Model size: {os.path.getsize(best_model_path) / (1024*1024):.1f} MB")
else:
    print("No trained model found. Complete training first.")

## Summary

After running this notebook, you will have:

1. **Trained YOLOv8 model** for trash detection
2. **Model weights** saved at: `models/TrashCAN_YoloV8_Jupyter/trashcan_detection/weights/best.pt`
3. **Training plots** showing loss curves and metrics
4. **Trash-only detector** class for filtering only trash detections
5. **Exported model** in ONNX format for deployment

### Next Steps:
- Use the trained model in your main application
- Integrate with your existing `inference.py`
- Deploy for real-time trash detection

### Model Performance:
The model will detect 14 different types of trash:
- `trash_bag`, `trash_bottle`, `trash_can`, `trash_container`
- `trash_clothing`, `trash_cup`, `trash_net`, `trash_pipe`
- `trash_rope`, `trash_snack_wrapper`, `trash_tarp`
- `trash_branch`, `trash_wreckage`, `trash_unknown_instance`